In [1]:
from crowd_sim import *
import gym

In [2]:
import matplotlib.pyplot as plt
%matplotlib qt

In [3]:
from arguments import get_args
from crowd_nav.configs.config import Config

In [4]:
env_id = 'CrowdSim-v0'
# env_id = 'CrowdSimVarNum-v0'
# env_id = 'CrowdSimSgan-v0'

env = gym.make(env_id)

In [5]:
env_config = Config()
env.configure(env_config)

In [6]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.set_xlabel('x(m)', fontsize=16)
ax.set_ylabel('y(m)', fontsize=16)
plt.ion()
plt.show()

In [7]:
env.setup(seed=1, num_of_env=1, ax=ax)

In [8]:
obs = env.reset()

In [12]:
env.render()

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(env, warn=True)